# Toxic Comment Classification (redone)
 This notebook will explore how to classify texts using a naive bayes classifier. A lot of inspiration is from Jermey Howard's kernel from the Kaggle's forum. Which also draws from the paper [Baselines and Bigrams: Simple, Good Sentiment and Topic Classification](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf). This journal is to replicate the results to show the strength of strong variants of this timeless classification model. In this instance to classify toxic comments. 

## The Problem: Given a text string, detect if its toxic.

We define toxic as a string with negative connotation that is detrimental to an intregity of a conversation. This can come in forms of insults, obscenities, threats, and identity hate. 

We are going use a dataset of wikipedia comments provided by Kaggle and train it on a learning model to a solid accuracy where we can predict if a comment is toxic or not.



## Exploratory analysis of our data

### Load the Dataset
Our first set is of course load the data! You will have to have the csv files in the same folder as this jupyter-notebook. To import it we will use the pandas library and represent it as a dataframe for easy table manipulation.

In [1]:
import pandas as pd

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

A peek at our training set. (***Warning for row 6***)

In [5]:
train_data.head(7)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0


In [6]:
test_data.head(7)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \r\r\n\r\r\n It says it right there that it ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \r\r\n\r\r\n == Before adding a new product ...",0,0,0,0,0,0
5,000663aff0fffc80,this other one from 1897,0,0,0,0,0,0
6,000689dd34e20979,== Reason for banning throwing == \r\r\n\r\r\n...,0,0,0,0,0,0


In [5]:
train_data["comment_text"][7]

"Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned."

In [3]:
type(train_data['toxic'][0])

numpy.int64

### Our Dataframe

We have 8 columns which are:
    - id - str
    - comment_text - str
    - toxic - numpy.int64
    - severe_toxic - numpy.int64
    - obscene - numpy.int64
    - threat - numpy.int64
    - insult - numpy.int64
    - identity_hate - numpy.int64

In this module we are going to drop the id column as there is no effect on the quality of the text or our analysis.

In [9]:
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [10]:
train_data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [13]:
test_data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,:Dear god this site is horrible.,0,0,0,0,0,0
2,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,""" \r\r\n\r\r\n It says it right there that it ...",0,0,0,0,0,0
4,""" \r\r\n\r\r\n == Before adding a new product ...",0,0,0,0,0,0


### How much data do we have?


In [22]:
LENGTH_TRAIN_DATA = len(train_data)
LENGTH_TEST_DATA = len(test_data)
LENGTH_ALL_DATA = length_train_data + length_test_data

In [23]:
print("Length of training data =",LENGTH_TRAIN_DATA)
print("Length of testing data =", LENGTH_TEST_DATA)
print("Total data is", LENGTH_ALL_DATA,"entries")

Length of training data = 159571
Length of testing data = 63978
Total data is 223549 entries


In [24]:
lengths = train_data.comment_text.str.len()
print(lengths.head())

#Average, standard deviation, max length
lengths.mean(),lengths.std(), lengths.max()

0    264
1    112
2    233
3    622
4     67
Name: comment_text, dtype: int64


(394.0732213246768, 590.7202819048923, 5000)

In [6]:
lengths.hist(label="Length")

## Amount of toxic comments 